<a href="https://colab.research.google.com/github/ciepielajan/GeoFabrik_Data_Extract/blob/main/GeoFabrik_raport_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import folium
import pandas as pd
from plotnine import *
!pip -q install geopandas
from geopandas import GeoDataFrame

%matplotlib inline

     |████████████████████████████████| 1.0MB 4.4MB/s 
     |████████████████████████████████| 6.5MB 11.5MB/s 
     |████████████████████████████████| 14.8MB 278kB/s 


In [101]:
link = "http://download.geofabrik.de/europe/poland/malopolskie-latest-free.shp.zip"

In [80]:
!wget $link  -O map.shp.zip

--2021-04-06 07:35:53--  http://download.geofabrik.de/europe/poland/malopolskie-latest-free.shp.zip
Resolving download.geofabrik.de (download.geofabrik.de)... 88.99.142.44, 116.202.112.212, 138.201.219.183
Connecting to download.geofabrik.de (download.geofabrik.de)|88.99.142.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232350136 (222M) [application/zip]
Saving to: ‘map.shp.zip’

map.shp.zip         100%[===================>] 221.59M  28.9MB/s    in 8.5s    

2021-04-06 07:36:02 (26.2 MB/s) - ‘map.shp.zip’ saved [232350136/232350136]



In [3]:
!unzip -q map.shp.zip -d /content/maps

In [4]:
from os import listdir

files_dir =  listdir("/content/maps")
files_list = []
for names in files_dir:
    if names.endswith(".shp"):
        files_list.append(names)
        
files_list = sorted(files_list)

#podgląd
print(pd.Series(files_list))

0     gis_osm_buildings_a_free_1.shp
1       gis_osm_landuse_a_free_1.shp
2       gis_osm_natural_a_free_1.shp
3         gis_osm_natural_free_1.shp
4        gis_osm_places_a_free_1.shp
5          gis_osm_places_free_1.shp
6          gis_osm_pofw_a_free_1.shp
7            gis_osm_pofw_free_1.shp
8          gis_osm_pois_a_free_1.shp
9            gis_osm_pois_free_1.shp
10       gis_osm_railways_free_1.shp
11          gis_osm_roads_free_1.shp
12      gis_osm_traffic_a_free_1.shp
13        gis_osm_traffic_free_1.shp
14    gis_osm_transport_a_free_1.shp
15      gis_osm_transport_free_1.shp
16        gis_osm_water_a_free_1.shp
17      gis_osm_waterways_free_1.shp
dtype: object


In [5]:
data_frames = {}
for name in files_list:
  name_df = name.split(".")[0]
  data_frames[name_df] = GeoDataFrame.from_file(f'maps/{name}', encoding='utf-8')

In [6]:
#podgląd
data_frames.keys()

dict_keys(['gis_osm_buildings_a_free_1', 'gis_osm_landuse_a_free_1', 'gis_osm_natural_a_free_1', 'gis_osm_natural_free_1', 'gis_osm_places_a_free_1', 'gis_osm_places_free_1', 'gis_osm_pofw_a_free_1', 'gis_osm_pofw_free_1', 'gis_osm_pois_a_free_1', 'gis_osm_pois_free_1', 'gis_osm_railways_free_1', 'gis_osm_roads_free_1', 'gis_osm_traffic_a_free_1', 'gis_osm_traffic_free_1', 'gis_osm_transport_a_free_1', 'gis_osm_transport_free_1', 'gis_osm_water_a_free_1', 'gis_osm_waterways_free_1'])

In [104]:
region = " - ".join(link.split("http://download.geofabrik.de/")[1].split("-latest")[0].split("/")).upper()

raport = ""
raport += f"# Raport region: {region}"
raport += "\n\n"

for name in data_frames:
  title = name.split("gis_osm_")[1].split("_free_1")[0].upper()
  raport += f"## {title}"
  raport += "\n"
  raport += f"Nazwa plik: {name}.shp"
  raport += "\n\n"
  raport += f"Ilość rekordów: {data_frames.get(name).shape[0]} "
  raport += "\n\n"
  raport += "Sample index "
  raport += "\n"
  raport += data_frames.get(name).sample(1).to_markdown()
  raport += "\n\n"
  raport += "Value counts for fclass"
  raport += "\n"
  raport += data_frames.get(name)["fclass"].value_counts().to_markdown()
  raport += "\n\n"

raport

'# Raport region: EUROPE - POLAND - MALOPOLSKIE\n\n## BUILDINGS_A\nNazwa plik: gis_osm_buildings_a_free_1.shp\n\nIlość rekordów: 1349206 \n\nSample index \n|        |    osm_id |   code | fclass   | name   | type   | geometry                                                                                                                   |\n|-------:|----------:|-------:|:---------|:-------|:-------|:---------------------------------------------------------------------------------------------------------------------------|\n| 685074 | 392589632 |   1500 | building |        |        | POLYGON ((21.0339745 49.9985188, 21.034015 49.9985936, 21.0341334 49.998567, 21.034093 49.9984922, 21.0339745 49.9985188)) |\n\nValue counts for fclass\n|          |      fclass |\n|:---------|------------:|\n| building | 1.34921e+06 |\n\n## LANDUSE_A\nNazwa plik: gis_osm_landuse_a_free_1.shp\n\nIlość rekordów: 182468 \n\nSample index \n|       |    osm_id |   code | fclass   | name   | geometry           

In [111]:
file_name= region.lower().replace(" ","")
f = open(f"raport-{file_name}.md", "a")
f.write(raport)
f.close()